In [4]:

import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)


Using Pytorch version :  1.9.0  Device :  cuda


In [25]:
#CIFAR10 데이터를 이용한 RESNET 설계
#1. 하이퍼 파라미터
BATCH_SIZE =  32 #한 배치당 32개 이미지
EPOCHS = 30 # 전체 데이터 셋을 10번 반복

In [7]:
# 2. 데이터 다운로드. Train Set, Test set 분리

train_dataset = datasets.CIFAR10(root="../data/CIFAR_10",
                               train=True,
                               download=True,
                               transform = transforms.ToTensor())
#train_set download. ToTensor()로 255스칼라값을 데이터를 0~1로 정규화.
#과적화 방지를 위해.
test_dataset = datasets.CIFAR10(root="../data/CIFAR_10",
                               train=False,
                               download=True,
                               transform = transforms.ToTensor())
#DATA LOADER 함수가 BATCH_size 단위로 분리해 지정.
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True) # 순서가 암기되는것을 막기위해.
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

Files already downloaded and verified
Files already downloaded and verified


In [20]:
#RESNET 18 ?
#계산하면 14. no bottleneck.
#cifar-10 데이터라서, 작게 쌓은것으로 생각.
class BasicBlock(nn.Module):
    # 반복적으로 이용되는 block
    def __init__(self, in_planes,planes,stride=1):
        super(BasicBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_planes,planes,
                               kernel_size=3,
                               stride = stride,# 첫번째 레이어만 stride를 가변적으로
                               padding=1,
                               bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes,planes,
                               kernel_size=3,
                               stride = 1,#여기서는 이미지 사이즈 그대로 유지
                               padding=1,
                               bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        #identity map. 크기가 같을 때.
        self.shortcut = nn.Sequential()#Skip Connection 정의
        
        #만약, stride가 달라지고, 인풋과 아웃풋이 크기가 다르면, Projection하는 개념.
        #stride가 달라져도, 한번만 conv해주면 된다. 
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                in_planes,planes, # 더하려면 depth가 같아야해서 conv2와 output사이즈를 같게 
                kernel_size = 1, #1x1로 데이터는 그대로 유지하고 합.
                bias=False,
                stride=stride
                ),
                nn.BatchNorm2d(planes)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # skip connection
        out = F.relu(out)
        return out
    

class ResNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ResNet,self).__init__()
        self.in_planes = 16
        self.conv1=nn.Conv2d(3,16, # 3채널 16 피처맵으로 시작
                             kernel_size=3,
                             stride = 1,
                             padding = 1,
                             bias = False
                            )
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16,2,stride=1)#16피처맵  레이어 4개
        self.layer2 = self._make_layer(32,2,stride=2)#32피처맵  레이어 4개
        self.layer3 = self._make_layer(64,2,stride=2)#32피처맵  레이어 4개
        self.linear = nn.Linear(64,num_classes)
            
    def _make_layer(self,planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes,planes,stride))
            self.in_planes=planes#출력결과로 반영
        return nn.Sequential(*layers)
    
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x))) #첫번째 레이어
        out = self.layer1(out)# 2~5
        out = self.layer2(out)# 6~9
        out = self.layer3(out)# 10~13
        out = F.avg_pool2d(out,8)
        out = out.view(out.size(0),-1)#데이터 펴기
        out = self.linear(out)#데이터 분류
        return out
        

In [21]:
#Optimizer, Objective Function
model = ResNet().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()
#원핫 인코딩값의 loss는 crossEntropyLoss로 비교
print(model)


ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [26]:
#8. 학습
def train(model,train_loader,optimizer, log_interval):
    model.train()
    for batch_idx,(image,label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        #데이터들 장비에 할당
        optimizer.zero_grad() # device 에 저장된 gradient 제거
        output = model(image) # model로 output을 계산
        loss = criterion(output, label) #loss 계산
        loss.backward() # loss 값을 이용해 gradient를 계산
        optimizer.step() # Gradient 값을 이용해 파라미터 업데이트.        

In [27]:
#9. 학습 진행하며, test 데이터로 모델 성능확인
def evaluate(model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            #print(output.max(1,keepdim=True))
            #print(prediction)
            correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
            #true.false값을 sum해줌. item
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss,test_accuracy
            

In [28]:
#10. 학습 및 평가.
for Epoch in range(1,EPOCHS+1):
    train(model,train_loader,optimizer,log_interval=200)
    test_loss,test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH:{}], \tTest Loss:{:.4f} \tTest Accuracy: {:.2f} %\n".
          format(Epoch,test_loss,test_accuracy))


[EPOCH:1], 	Test Loss:0.0188 	Test Accuracy: 80.42 %


[EPOCH:2], 	Test Loss:0.0184 	Test Accuracy: 81.96 %


[EPOCH:3], 	Test Loss:0.0184 	Test Accuracy: 82.04 %


[EPOCH:4], 	Test Loss:0.0190 	Test Accuracy: 82.19 %


[EPOCH:5], 	Test Loss:0.0197 	Test Accuracy: 81.75 %


[EPOCH:6], 	Test Loss:0.0199 	Test Accuracy: 81.73 %


[EPOCH:7], 	Test Loss:0.0220 	Test Accuracy: 81.07 %


[EPOCH:8], 	Test Loss:0.0233 	Test Accuracy: 80.17 %


[EPOCH:9], 	Test Loss:0.0212 	Test Accuracy: 81.98 %


[EPOCH:10], 	Test Loss:0.0256 	Test Accuracy: 80.08 %


[EPOCH:11], 	Test Loss:0.0240 	Test Accuracy: 81.11 %


[EPOCH:12], 	Test Loss:0.0230 	Test Accuracy: 81.36 %


[EPOCH:13], 	Test Loss:0.0255 	Test Accuracy: 81.07 %


[EPOCH:14], 	Test Loss:0.0258 	Test Accuracy: 81.05 %


[EPOCH:15], 	Test Loss:0.0245 	Test Accuracy: 81.54 %


[EPOCH:16], 	Test Loss:0.0252 	Test Accuracy: 81.37 %


[EPOCH:17], 	Test Loss:0.0254 	Test Accuracy: 81.33 %


[EPOCH:18], 	Test Loss:0.0275 	Test Accuracy: 80.61 %


